## Analisis de Vecindarios de Toronto City

In [35]:
import requests # librería para manejar las solicitudes
import pandas as pd # librería para análisis de datos
import numpy as np # librería para manejar datos vectorizados
import random # librería para generar números aleatorios

!pip install geopy
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 

# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# librería para convertir un archivo json en un dataframe pandas
from pandas.io.json import json_normalize

#! pip install folium==0.5.0
##! pip install geocoder

import folium # librería para graficar 

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
direccion_Toronto = 'Toronto City, CA'

geolocalizar_To = Nominatim(user_agent='to_explorador')
localazacion_to = geolocalizar_To.geocode(direccion_Toronto)

lat_to = localazacion_to.latitude
lng_to = localazacion_to.longitude

print('La latitud y la longitud de Toronto City es: {} y {} '.format(lat_to,lng_to))


La latitud y la longitud de Toronto City es: 43.6534817 y -79.3839347 


In [39]:
#importar los datos de origen en un dataframe
Canada_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
Canada_df.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [40]:
#convertir en una sola columna
Canada_df = Canada_df.melt()
Canada_df.head()

,variable,value
0,0,M1ANot assigned
1,0,M1BScarborough(Malvern / Rouge)
2,0,M1CScarborough(Rouge Hill / Port Union / Highl...
3,0,M1EScarborough(Guildwood / Morningside / West ...
4,0,M1GScarborough(Woburn)


In [42]:
import re
#separar el codigo postal obteniendo las 3 primeras letras del contenido
Canada_df['Postal Code'] = Canada_df['value'].apply(lambda x: x[:3])

#el resto del contenido ponerlo temporalmente como municipio
Canada_df['Municipio'] = Canada_df['value'].apply(lambda x: x[3:])

#no considerar los registros que no tienen municipio
Canada_df = Canada_df[Canada_df['Municipio']!='Not assigned'].reset_index(drop=True)

#separa el nombre del municipio a aprtir de encontrar un parentesis y el resultado ponerlo como barrio
Canada_df['Barrio'] = Canada_df['Municipio'].apply(lambda x: (re.findall("\((.*?)\)", x))[0])

#separa el nombre del barrio que esta aun en la columna municipio desde el inicio hasta el parentesis 
Canada_df['Municipio'] = Canada_df['Municipio'].apply(lambda x: x.split('(')[0])

#reemplazar la barra / con una coma ,
Canada_df['Barrio'] = Canada_df['Barrio'].apply(lambda x: x.replace(" / ",','))

#Seleccionar solo las 3 columnas solicitadas
Canada_df_final = Canada_df[['Postal Code','Municipio','Barrio']]
Canada_df_final.head()

,Postal Code,Municipio,Barrio
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [43]:
Canada_df_final.shape

(103, 3)

## Hasta aqui la primera parte de la tarea

In [44]:
#cargar el archivo de coordenadas proporcinado
coordenadas = pd.read_csv('Geospatial_Coordinates.csv')
coordenadas.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
#Obtener un solo dataframe con los datos de los barrios y las coordenadas
BarriosCoordenadas = pd.merge(Canada_df_final, coordenadas, on = 'Postal Code', how='inner')
BarriosCoordenadas.head()


,Postal Code,Municipio,Barrio,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Hasta aqui la segunda parte de la tarea

In [46]:
#Obtener solo los datos de TORONTO
Toronto_df = BarriosCoordenadas[BarriosCoordenadas['Municipio'].str.contains('oronto')]
Toronto_df.head()

,Postal Code,Municipio,Barrio,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [32]:
Toronto_map = folium.Map(location=[lat_to, lng_to], zoom_start=13) # generar el mapa alrededor de TORONTO

# Añadir TORONTO como una marca circular roja
folium.CircleMarker(
    [lat_to, lng_to],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(Toronto_map)

# añadir los barrios al mapa como marcas circulares azules
for lat, lng, label in zip(Toronto_df.Latitude, Toronto_df.Longitude, Toronto_df.Barrio):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(Toronto_map)

# mostrar mapa
Toronto_map

In [47]:
Toronto_df.groupby("Municipio").size().sort_values(ascending=False)

Municipio
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

## Hasta aqui la Tercera parte de la tarea